# 🧠 Persistencia de Modelos en Machine Learning

En este cuaderno exploramos diferentes técnicas para **guardar** y **recuperar** modelos de Machine Learning en Python.  
La persistencia de modelos es un paso clave en cualquier pipeline de **Machine Learning** o **Data Science**, ya que nos permite reutilizar modelos entrenados sin tener que volver a entrenarlos cada vez.

---

## ✨ Objetivos
- Comprender la importancia de la persistencia de modelos.
- Aprender a usar **Pickle** y **Joblib** para serializar objetos de Python.
- Implementar clases personalizadas con métodos de **exportación a JSON**.
- Comparar ventajas y limitaciones de cada método.

---

## 📦 Métodos de persistencia

1. **Pickle**
   - Librería estándar de Python.
   - Serializa objetos en archivos binarios.
   - Útil para almacenar tuplas con modelos, datasets y métricas.

2. **Joblib**
   - Optimizado para objetos grandes como modelos de Scikit-Learn.
   - Recomendado para trabajar con arreglos de NumPy y objetos pesados.

3. **JSON**
   - No guarda directamente modelos, pero es útil para exportar **parámetros** y **datasets pequeños** en formato legible.
   - Ideal para configuraciones, hiperparámetros y reproducibilidad.

---

## 🔑 ¿Por qué es importante?
- Acelera la **implementación en producción**.
- Permite **compartir modelos** entre equipos.
- Facilita la **reproducibilidad de experimentos**.
- Ahorra **tiempo y recursos computacionales**.

---


In [2]:
# https://www.analyticslane.com/2021/11/22/guardar-los-modelos-de-scikit-learn-en-disco-e-importarlo-en-otra-sesion/
# http://exponentis.es/persistencia-de-modelos-en-python-como-guardar-tu-modelo-entrenado-de-machine-learning
# https://www.datasmarts.net/como-guardar-y-cargar-modelos-de-machine-learning-en-scikit-learn/
# https://soloelectronicos.com/2021/10/01/scikit-learn-guardar-y-restaurar-modelos/   agrega json

# Importamos las librerías necesarias
import pickle
import joblib
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier


In [3]:
# Cargamos el conjunto de datos.
X, y = load_breast_cancer(return_X_y=True)



In [4]:
# Instanciamos un clasificador Naïve Bayes Gaussiano.
classifier = DecisionTreeClassifier()

# Entrenamos el modelo.
print('Entrenando modelo...')
classifier.fit(X, y)

Entrenando modelo...


DecisionTreeClassifier()

In [5]:
print(classifier.score(X, y))

1.0


In [6]:
# Guardamos usando pickle...
print('Guardando modelo usando pickle...')
with open('model.pickle', 'wb') as f:
    pickle.dump(classifier, f)

# ... y ahora usando joblib
print('Guardando modelo usando joblib...')
joblib.dump(classifier, 'model.joblib')

# se puede usar el parámetro compress para 'encoger' el modelo
# joblib.dump(model, 'compress.joblib', compress=9)

Guardando modelo usando pickle...
Guardando modelo usando joblib...


['model.joblib']

In [7]:
# Cargamos el modelo que guardamos con pickle...
print('Cargando modelo usando pickle...')
with open('model.pickle', 'rb') as f:
    classifier_pickle = pickle.load(f)

# ... Calculamos el accuracy del modelo sobre los datos, e imprimimos su tipo.
print(f'[PICKLE] Tipo del modelo: {type(classifier_pickle)}')
print(f'[PICKLE] Accuracy del modelo: {classifier_pickle.score(X, y) * 100:.2f}%')


Cargando modelo usando pickle...
[PICKLE] Tipo del modelo: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
[PICKLE] Accuracy del modelo: 100.00%


In [8]:
# Cargamos el modelo que guardamos con joblib...
print('Cargando modelo usando joblib...')
classifier_joblib = joblib.load('model.joblib')

# ... Calculamos el accuracy del modelo sobre los datos, e imprimimos su tipo.
print(f'[JOBLIB] Tipo del modelo: {type(classifier_joblib)}')
print(f'[JOBLIB] Accuracy del modelo: {classifier_joblib.score(X, y) * 100:.2f}%')

Cargando modelo usando joblib...
[JOBLIB] Tipo del modelo: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
[JOBLIB] Accuracy del modelo: 100.00%


In [9]:
# Ejemplo 2 además json

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle
import joblib
import json
import numpy as np


In [10]:

# Load and split data
data = load_iris()
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data.data, data.target, test_size=0.3, random_state=4)

# Create a model
model = DecisionTreeClassifier()
model.fit(Xtrain, Ytrain)

DecisionTreeClassifier()

In [11]:
print(model.score(Xtest, Ytest))

0.9777777777777777


In [12]:
# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

# Calculate the accuracy score and predict target values
score = pickle_model.score(Xtest, Ytest)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(Xtest)

Test score: 97.78 %


In [ ]:
tuple_objects = (model, Xtrain, Ytrain, score)  # Agrupa modelo, datos y score en una tupla

pickle.dump(tuple_objects, open("tuple_model.pkl", 'wb'))  # Guarda la tupla en un archivo binario

pickled_model, pickled_Xtrain, pickled_Ytrain, pickled_score = pickle.load(open("tuple_model.pkl", 'rb'))  # Restaura la tupla desde el archivo

In [ ]:
joblib_file = "joblib_model.pkl"              # Nombre del archivo donde se guardará el modelo
joblib.dump(model, joblib_file)               # Guarda (serializa) el modelo entrenado en un archivo

# Load from file
joblib_model = joblib.load(joblib_file)       # Carga (deserializa) el modelo guardado

# Calculate the accuracy and predictions
score = joblib_model.score(Xtest, Ytest)      # Calcula el accuracy del modelo con datos de prueba
print("Test score: {0:.2f} %".format(100 * score))  # Muestra el resultado en porcentaje
Ypredict = pickle_model.predict(Xtest)  # Predice con el modelo cargado

Test score: 97.78 %


In [ ]:
class MyTree(DecisionTreeClassifier):
    # Constructor que hereda correctamente de DecisionTreeClassifier
    def __init__(self, min_samples_leaf=1, criterion='gini', max_depth=None, X_train=None, Y_train=None):
        # Inicializa el DecisionTreeClassifier con los parámetros dados
        DecisionTreeClassifier.__init__(self, 
                                        criterion=criterion, 
                                        max_depth=max_depth, 
                                        min_samples_leaf=min_samples_leaf)
        # Guarda opcionalmente los datos de entrenamiento
        self.X_train = X_train
        self.Y_train = Y_train

    # Método para guardar el objeto en un archivo JSON
    def save_json(self, filepath):
        dict_ = {
            'max_depth': self.max_depth,
            'criterion': self.criterion,
            'min_samples_leaf': self.min_samples_leaf,
            'X_train': self.X_train.tolist() if self.X_train is not None else 'None',
            'Y_train': self.Y_train.tolist() if self.Y_train is not None else 'None'
        }
        # Convierte el diccionario a JSON y lo guarda
        json_txt = json.dumps(dict_, indent=4)
        with open(filepath, 'w') as file:
            file.write(json_txt)

    # Método para cargar los datos desde un archivo JSON
    def load_json(self, filepath):
        with open(filepath, 'r') as file:
            dict_ = json.load(file)

        # Restaura los parámetros y datos
        self.criterion = dict_['criterion']
        self.max_depth = dict_['max_depth']
        self.min_samples_leaf = dict_['min_samples_leaf']
        self.X_train = np.asarray(dict_['X_train']) if dict_['X_train'] != 'None' else None
        self.Y_train = np.asarray(dict_['Y_train']) if dict_['Y_train'] != 'None' else None

In [16]:
filepath = "mytree.json"

# Create a model and train it
mylogreg = MyTree(X_train=Xtrain, Y_train=Ytrain)
mylogreg.save_json(filepath)

# Create a new object and load its data from JSON file
json_mytree = MyTree()
json_mytree.load_json(filepath)
json_mytree

MyTree(X_train=array([[4.3, 3. , 1.1, 0.1],
       [5.7, 4.4, 1.5, 0.4],
       [5.9, 3. , 4.2, 1.5],
       [6.1, 3. , 4.6, 1.4],
       [6.5, 3. , 5.5, 1.8],
       [5.2, 3.5, 1.5, 0.2],
       [5.6, 2.5, 3.9, 1.1],
       [7.7, 2.6, 6.9, 2.3],
       [6.3, 3.4, 5.6, 2.4],
       [6.2, 2.9, 4.3, 1.3],
       [5.7, 2.9, 4.2, 1.3],
       [5. , 3.5, 1.6, 0.6],
       [5.6, 2.9, 3.6, 1.3],
       [6. , 2.2, 5. , 1.5],
       [5.5, 2.6, 4.4, 1.2],
       [4.6, 3.4, 1.4, 0.3],
       [5.6, 3. , 4.1, 1.3],
       [5.1, 3.4, 1.5, 0.2],
       [6.4, 2.9, 4.3...
       [6.6, 2.9, 4.6, 1.3],
       [6.4, 3.1, 5.5, 1.8],
       [7. , 3.2, 4.7, 1.4],
       [6.3, 2.3, 4.4, 1.3],
       [6.5, 3. , 5.8, 2.2],
       [7.2, 3. , 5.8, 1.6],
       [7.7, 2.8, 6.7, 2. ]]),
       Y_train=array([0, 0, 1, 1, 2, 0, 1, 2, 2, 1, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2,
       1, 0, 2, 2, 0, 1, 2, 0, 2, 1, 2, 1, 0, 2, 1, 2, 0, 2, 1, 2, 1, 2,
       1, 1, 2, 1, 1, 2, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 2, 2, 1, 1,
       1, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 0, 0, 2, 1, 0, 0, 2,
       1, 2, 0, 0, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2]))

In [17]:
# Since the custom load_json method only loads the data and not the fitted state
# of the scikit-learn model, we need to fit the model after loading.

json_mytree.fit(json_mytree.X_train, json_mytree.Y_train)

print(json_mytree.score(Xtest, Ytest))

0.9777777777777777
